In [1]:
# Image Poisoning Demo with Gradio

##This notebook creates an interactive interface for image poisoning using Gradio.

In [2]:
# Import required libraries
import gradio as gr
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import cv2
from scipy.ndimage import gaussian_filter
import torch
import torch.nn.functional as F

/opt/anaconda3/envs/my_hemlock_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-03 19:24:04.559730: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load the pre-trained model
model = ResNet50(weights='imagenet')

In [4]:
def load_and_preprocess_image(image):
    """Load and preprocess an image for the model."""
    # Convert to PIL Image if it's a numpy array
    if isinstance(image, np.ndarray):
        img = Image.fromarray(image)
    else:
        img = image
        
    # Resize image while maintaining aspect ratio
    target_size = (224, 224)
    img.thumbnail(target_size, Image.Resampling.LANCZOS)
    
    # Create a new image with padding
    new_img = Image.new('RGB', target_size, (0, 0, 0))
    offset = ((target_size[0] - img.size[0]) // 2,
              (target_size[1] - img.size[1]) // 2)
    new_img.paste(img, offset)
    
    # Convert to numpy array
    img_array = np.array(new_img)
    
    # Ensure the image is in RGB format
    if len(img_array.shape) == 2:  # Grayscale
        img_array = np.stack([img_array] * 3, axis=-1)
    elif img_array.shape[2] == 4:  # RGBA
        img_array = img_array[:, :, :3]
        
    return img_array

In [5]:
def create_adversarial_pattern(input_image, input_label, model, epsilon):
    """Create an adversarial pattern using FGSM with improved gradient computation."""
    with tf.GradientTape() as tape:
        tape.watch(input_image)
        prediction = model(input_image)
        loss = tf.keras.losses.sparse_categorical_crossentropy(input_label, prediction)
    
    # Get the gradients of the loss with respect to the input image
    gradient = tape.gradient(loss, input_image)
    
    # Apply gradient smoothing
    gradient = gaussian_filter(gradient.numpy(), sigma=1.0)
    gradient = tf.convert_to_tensor(gradient)
    
    # Get the sign of the gradients to create the perturbation
    signed_grad = tf.sign(gradient)
    
    # Create the adversarial pattern with improved scaling
    perturbation = epsilon * signed_grad
    
    return perturbation

In [6]:
def apply_perturbation(image, epsilon):
    """Apply perturbation to the image with improved preprocessing and scaling."""
    # Store original image for reference
    original = image.copy()
    
    # Preprocess the image for the model
    preprocessed = preprocess_input(image)
    preprocessed = np.expand_dims(preprocessed, axis=0)
    
    # Get the model's prediction
    predictions = model.predict(preprocessed)
    predicted_label = np.argmax(predictions[0])
    
    # Create adversarial pattern
    perturbation = create_adversarial_pattern(
        tf.convert_to_tensor(preprocessed),
        tf.convert_to_tensor([predicted_label]),
        model,
        epsilon
    )
    
    # Apply the perturbation with improved scaling
    perturbed = preprocessed + perturbation.numpy()
    
    # Convert back to original scale with proper denormalization
    perturbed = perturbed[0]  # Remove batch dimension
    perturbed = (perturbed + 1) * 127.5  # Denormalize from [-1, 1] to [0, 255]
    
    # Apply adaptive thresholding based on epsilon
    if epsilon > 0:
        threshold = 127.5 * epsilon
        perturbed = np.where(np.abs(perturbed - original) > threshold, perturbed, original)
    
    # Clip to valid range
    perturbed = np.clip(perturbed, 0, 255)
    
    return perturbed, predicted_label

In [7]:

        
def process_image(image, epsilon):
    """Process the image and return both original and perturbed versions."""
    try:
        # Load and preprocess the image
        image_array = load_and_preprocess_image(image)
        
        # Get original prediction
        preprocessed = preprocess_input(image_array.reshape(1, 224, 224, 3))
        original_pred = model.predict(preprocessed)
        original_label = decode_predictions(original_pred, top=1)[0][0][1]
        
        # Apply perturbation with epsilon scaling
        perturbed_image, _ = apply_perturbation(image_array, epsilon)
        
        # Get perturbed prediction
        preprocessed_perturbed = preprocess_input(perturbed_image.reshape(1, 224, 224, 3))
        perturbed_pred = model.predict(preprocessed_perturbed)
        perturbed_label = decode_predictions(perturbed_pred, top=1)[0][0][1]
        
        # Create figure with both images
        plt.figure(figsize=(15, 7))
        
        # Display original image
        plt.subplot(1, 2, 1)
        plt.imshow(image_array)
        plt.title(f"Original Image\nClass: {original_label}")
        plt.axis('off')
        
        # Display perturbed image
        plt.subplot(1, 2, 2)
        plt.imshow(perturbed_image.astype(np.uint8))
        plt.title(f"Perturbed Image (ε={epsilon:.2f})\nClass: {perturbed_label}")
        plt.axis('off')
        
        plt.tight_layout()
        
        # Save the figure to a temporary file
        plt.savefig('temp_result.png', bbox_inches='tight', pad_inches=0)
        plt.close()
        
        return 'temp_result.png'
        
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        return None

In [8]:
# Create the Gradio interface
iface = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(label="Upload an image"),
        gr.Slider(minimum=0.0, maximum=0.5, value=0.1, step=0.01, label="Perturbation Strength (ε)")
    ],
    outputs=gr.Image(label="Results"),
    title="Image Poisoning Demo",
    description="Upload an image and adjust the perturbation strength to see how it affects the model's classification.",
    examples=[
        ["example1.jpg", 0.1],
        ["example2.jpg", 0.2]
    ]
)

In [ ]:
#######################################Next Cell is Test Cell#######################################

In [10]:
# Cell 1: Importing Required Libraries and Setting Up the Model
# This cell imports all necessary libraries and loads the MobileNetV2 model
# with pre-trained weights from ImageNet.

# Import Gradio for creating the web interface
import gradio as gr
# Import TensorFlow for deep learning operations
import tensorflow as tf
# Import NumPy for numerical operations
import numpy as np
# Import PIL for image processing
from PIL import Image
# Import OpenCV for advanced image operations
import cv2
# Import MobileNetV2 model and its preprocessing functions
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
# Import Matplotlib for creating visualizations
import matplotlib.pyplot as plt

# Load the MobileNetV2 model with pre-trained weights
model = MobileNetV2(weights='imagenet')

# Cell 2: Image Preprocessing Function
# This function handles loading and preprocessing of input images
# to ensure they're in the correct format for the model.

def load_and_preprocess_image(image):
    """Load and preprocess an image while preserving original values."""
    # Create a copy of the input image if it's already a numpy array
    if isinstance(image, np.ndarray):
        img_array = image.copy()
    else:
        # Convert PIL image to numpy array
        img_array = np.array(image)
    
    # Convert grayscale images to RGB by repeating the channel
    if len(img_array.shape) == 2:
        img_array = np.stack([img_array] * 3, axis=-1)
    # Remove alpha channel if present
    elif img_array.shape[2] == 4:
        img_array = img_array[:, :, :3]
    
    # Resize image to 224x224 (required by MobileNetV2)
    return cv2.resize(img_array, (224, 224))

# Cell 3: Adversarial Example Generation
# This function creates adversarial examples using the Fast Gradient Sign Method (FGSM)
# by computing gradients of the loss with respect to the input image.

def generate_adversarial(image, epsilon):
    """Generate adversarial example using FGSM."""
    # Convert image to tensor and add batch dimension
    image_tensor = tf.convert_to_tensor(image)
    image_tensor = tf.cast(image_tensor, tf.float32)
    image_tensor = tf.expand_dims(image_tensor, 0)
    
    # Compute gradients using gradient tape
    with tf.GradientTape() as tape:
        # Watch the input tensor for gradient computation
        tape.watch(image_tensor)
        # Preprocess image for the model
        preprocessed = preprocess_input(image_tensor)
        # Get model prediction
        prediction = model(preprocessed, training=False)
        # Get the predicted class
        target_class = tf.argmax(prediction[0])
        target_class = tf.cast(target_class, tf.int64)
        # Compute loss
        loss = tf.keras.losses.SparseCategoricalCrossentropy()(
            tf.expand_dims(target_class, 0), prediction
        )
    
    # Get gradients of loss with respect to input
    gradients = tape.gradient(loss, image_tensor)
    # Create perturbation using sign of gradients
    perturbation = epsilon * tf.sign(gradients)
    # Apply perturbation to original image
    adversarial_image = image_tensor + perturbation
    # Clip values to valid range [0, 255]
    adversarial_image = tf.clip_by_value(adversarial_image, 0, 255)
    
    # Return perturbed image as numpy array
    return adversarial_image[0].numpy().astype(np.uint8)

# Cell 4: Visualization Creation
# This function creates a professional comparison visualization
# showing original image, perturbed image, and the difference between them.

def create_comparison_visualization(original, perturbed, orig_label, pert_label, orig_conf, pert_conf, epsilon):
    """Create a professional comparison visualization."""
    # Set dark theme for better visibility
    plt.style.use('dark_background')
    # Create figure with specific size
    fig = plt.figure(figsize=(15, 6))
    
    # Create grid for subplots
    gs = plt.GridSpec(1, 3, width_ratios=[1, 1, 1], wspace=0.3)
    
    # Plot original image
    ax1 = plt.subplot(gs[0])
    ax1.imshow(original)
    ax1.set_title(f"Original\n{orig_label}\nConfidence: {orig_conf:.1f}%", 
                 color='white', pad=10)
    ax1.axis('off')
    
    # Plot perturbed image
    ax2 = plt.subplot(gs[1])
    ax2.imshow(perturbed)
    ax2.set_title(f"Perturbed (ε={epsilon:.3f})\n{pert_label}\nConfidence: {pert_conf:.1f}%", 
                 color='white', pad=10)
    ax2.axis('off')
    
    # Plot difference visualization
    ax3 = plt.subplot(gs[2])
    # Compute absolute difference between images
    difference = cv2.absdiff(original, perturbed)
    # Enhance difference visualization using colormap
    difference = cv2.applyColorMap(
        cv2.convertScaleAbs(difference, alpha=5), 
        cv2.COLORMAP_VIRIDIS
    )
    ax3.imshow(cv2.cvtColor(difference, cv2.COLOR_BGR2RGB))
    ax3.set_title("Perturbation Map\n(Enhanced Difference)", 
                 color='white', pad=10)
    ax3.axis('off')
    
    # Set dark background
    fig.patch.set_facecolor('#1A1A1A')
    
    # Save figure with specific settings
    plt.savefig('temp_result.png', 
                bbox_inches='tight', 
                facecolor='#1A1A1A', 
                edgecolor='none', 
                pad_inches=0.2,
                dpi=150)
    plt.close()
    
    # Read and convert saved image
    result_img = cv2.imread('temp_result.png', cv2.IMREAD_UNCHANGED)
    return cv2.cvtColor(result_img, cv2.COLOR_BGR2RGB)

# Cell 5: Main Processing Function
# This function handles the entire image processing pipeline,
# from loading the image to generating the final visualization.

def process_image(image, epsilon):
    """Process image and return professional visualization."""
    try:
        # Load and preprocess input image
        original_image = load_and_preprocess_image(image)
        
        # Get prediction for original image
        orig_preprocessed = preprocess_input(original_image[np.newaxis, ...].astype(np.float32))
        orig_pred = model.predict(orig_preprocessed)
        # Format label and get confidence
        orig_label = decode_predictions(orig_pred, top=1)[0][0][1].replace('_', ' ').title()
        orig_conf = float(orig_pred.max()) * 100
        
        # Handle epsilon=0 case
        if epsilon == 0:
            perturbed_image = original_image.copy()
            pert_label = orig_label
            pert_conf = orig_conf
        else:
            # Generate perturbed image
            perturbed_image = generate_adversarial(original_image, epsilon)
            # Get prediction for perturbed image
            pert_preprocessed = preprocess_input(perturbed_image[np.newaxis, ...].astype(np.float32))
            pert_pred = model.predict(pert_preprocessed)
            # Format label and get confidence
            pert_label = decode_predictions(pert_pred, top=1)[0][0][1].replace('_', ' ').title()
            pert_conf = float(pert_pred.max()) * 100
        
        # Create final visualization
        result = create_comparison_visualization(
            original_image, perturbed_image,
            orig_label, pert_label,
            orig_conf, pert_conf,
            epsilon
        )
        
        return result
        
    except Exception as e:
        print(f"Error processing image: {str(e)}")
        return None

# Cell 6: Gradio Interface Creation
# This cell creates the web interface using Gradio,
# setting up the input and output components.

# Create the Gradio interface with a professional dark theme
iface = gr.Interface(
    fn=process_image,  # Main processing function
    inputs=[
        gr.Image(type="numpy", label="Upload Image"),  # Image upload component
        gr.Slider(minimum=0.0, maximum=0.1, value=0.0, step=0.005, 
                 label="Perturbation Strength (ε)")  # Epsilon slider
    ],
    outputs=gr.Image(type="numpy", label="Analysis Results"),  # Results display
    title="Advanced Adversarial Image Analysis",  # Interface title
    description="Upload an image and adjust the perturbation strength to visualize how it affects MobileNetV2's classification. The rightmost panel shows an enhanced visualization of the perturbation pattern.",
    theme="dark"  # Dark theme for better visibility
)

# Launch the interface
iface.launch()

/opt/anaconda3/envs/my_hemlock_env/lib/python3.12/site-packages/gradio/blocks.py:1115: UserWarning: Cannot load dark. Caught Exception: 404 Client Error: Not Found for url: https://huggingface.co/api/spaces/dark (Request ID: Root=1-67ef1b80-7dbae58741ce6474419075bb;1f4b8779-7c78-4ce4-9e31-51a23ba77e0c)

Sorry, we can't find the page you are looking for.
  warnings.warn(f"Cannot load {theme}. Caught Exception: {str(e)}")


* Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
